In [148]:
# you may need this packages when you first time run on your sagemaker instances

# %conda install psycopg2
# %conda install -c anaconda boto3

# %conda install -y -c anaconda sqlalchemy

In [2]:
import boto3
import json
import pandas as pd
import numpy as np
from datetime import timedelta
from sqlalchemy import create_engine

from IPython.display import display
import matplotlib.pyplot as plt

## Connect to Database

In [3]:
#personal tocken
dsn = dict(host='application-db-ro.test.arable.cloud',
           user='hong_tang',
           password='rsPq9J2GIEjq',
           port='5432',
           dbname='arable'
          )
sqlalchemy_dsn = 'postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**dsn)

In [4]:
def connect_db(dsn: str) -> str:
    cnx = create_engine(dsn)

    return cnx

## SQL Queries to access database

In [5]:
def read_hourly(cnx, device, begin, end):
    schema_raw = 'hourly'
    query_template_raw = """
select h.time, h.device, h.location, h.precip, h.rh 
from device_data.{schema} as h
where h.device = '{device}' and h.time >= '{start}' and h.time < '{end}';
"""

    sql_query = query_template_raw.format(schema=schema_raw, device=device, start=begin, end=end)
    df = pd.read_sql_query(sql_query, cnx)

    return df

def read_raw(cnx, device, begin, end):
    schema_raw = 'raw'
    query_template_raw = """
select r.time, r.device, r.location, r.rh 
from device_data.{schema} as r
where r.device = '{device}' and r.time >= '{start}' and r.time < '{end}';
"""

    sql_query = query_template_raw.format(schema=schema_raw, device=device, start=begin, end=end)

    df = pd.read_sql_query(sql_query, cnx)

    return df

## Function to add time as index to final dataframe

In [6]:
def settimeindex(df_final):
    '''set time as time index for later plot and data management'''
    df_final=df_final.reset_index()
    if 'time' in df_final.columns:
        df_final.set_index(pd.DatetimeIndex(df_final.time), inplace=True)
    elif 'ref_hourlydate' in df_final.columns:
        df_final.set_index(pd.DatetimeIndex(df_final.ref_hourlydate), inplace=True)
    return df_final


## initialize database connection

In [7]:
pg_conn = connect_db(sqlalchemy_dsn)

## Starting to select devices you want to pull..

### Example 1. pull CALVAL Data

In [8]:
device_list=['C005252', 'C004146']
start='2021-05-21'
end='2021-06-17'

In [61]:

def read_calval(cnx, device, begin, end):
    schema_calval = 'calval_ref_data'
    query_template_calval = """

select crd.ref_hourlydate, cdr.mark2 device, 
avg(crd.ref_tair) ref_tair,avg(crd.ref_p) ref_p,
avg(crd.ref_swdw) ref_swdw,avg(crd.ref_tdew) ref_tdew,
avg(crd.ref_swuw) ref_swuw,avg(crd.ref_lwdw) ref_lwdw,
avg(crd.ref_lwuw) ref_lwuw,avg(crd.ref_lfw) ref_lfw,
avg(crd.ref_tbelow) ref_tbelow,avg(crd.ref_pardw) ref_pardw,
avg(crd.ref_paruw) ref_paruw,avg(crd.ref_lat) ref_lat,avg(crd.ref_long) ref_long,
avg(crd.ref_rh) ref_rh,
sum(crd.ref_precip) ref_precip
from device_data.{schema} crd 
join
	 device_data.calvaldevicereference cdr 
	 on crd.site_id=cdr.site
where crd.ref_hourlydate between '{start}' and '{end}'
and crd.ref_precip is not null
and cdr.mark2 = '{device}'

group by crd.ref_hourlydate,cdr.mark2
order by cdr.mark2 asc, 
		crd.ref_hourlydate asc
--limit 10

"""

    sql_query = query_template_calval.format(schema=schema_calval, device=device, start=begin, end=end)

    df = pd.read_sql_query(sql_query, cnx)

    return df

In [62]:
device_list=['C004223', 'C005042'] 
start='2021-05-21'
end='2021-06-17'

In [63]:
series = []
df_final = pd.DataFrame()

for device in device_list:
    print(device, start, end)
#     device='*'
    df_calval = read_calval(pg_conn, device, start, end)
    print(df_calval.shape)
    if not df_calval.empty:
        df_final=pd.concat([df_final, df_calval])


C004223 2021-05-21 2021-06-17
(649, 17)
C005042 2021-05-21 2021-06-17
(649, 17)


In [64]:
#change set time data as index, run 
df_calval=settimeindex(df_final)

In [65]:
df_calval

,index,ref_hourlydate,device,ref_tair,ref_p,ref_swdw,ref_tdew,ref_swuw,ref_lwdw,ref_lwuw,ref_lfw,ref_tbelow,ref_pardw,ref_paruw,ref_lat,ref_long,ref_rh,ref_precip
ref_hourlydate,,,,,,,,,,,,,,,,,,
2021-05-21 00:00:00+00:00,0,2021-05-21 00:00:00+00:00,C004223,22.444444,100.824362,185.583333,20.638108,None,None,None,None,None,None,None,None,None,0.895833,2.794
2021-05-21 01:00:00+00:00,1,2021-05-21 01:00:00+00:00,C004223,22.129629,100.778644,110.750000,20.496872,None,None,None,None,None,None,None,None,None,0.905000,0.254
2021-05-21 02:00:00+00:00,2,2021-05-21 02:00:00+00:00,C004223,22.638888,100.768202,96.000000,20.407631,None,None,None,None,None,None,None,None,None,0.872500,0.000
2021-05-21 03:00:00+00:00,3,2021-05-21 03:00:00+00:00,C004223,22.787038,100.773000,64.250000,20.003992,None,None,None,None,None,None,None,None,None,0.843333,0.000
2021-05-21 04:00:00+00:00,4,2021-05-21 04:00:00+00:00,C004223,22.583333,100.818153,18.500000,19.773106,None,None,None,None,None,None,None,None,None,0.841667,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-16 20:00:00+00:00,644,2021-06-16 20:00:00+00:00,C005042,24.379629,101.024722,752.083333,21.655668,None,None,None,None,None,None,None,None,None,0.850000,0.000
2021-06-16 21:00:00+00:00,645,2021-06-16 21:00:00+00:00,C005042,25.106482,100.986060,375.500000,21.270362,None,None,None,None,None,None,None,None,None,0.793333,0.000
2021-06-16 22:00:00+00:00,646,2021-06-16 22:00:00+00:00,C005042,25.532408,100.945989,494.666667,21.027987,None,None,None,None,None,None,None,None,None,0.762500,0.000


## Here We learn how to pull CalVal Data, Good Job!

## EXample 2: pull CalVal-Mark2 Joined Data 

### Currently only hourly data are available. We are working on pull 5min data!

In [75]:

def read_calval_dev(cnx, device, begin, end):
    schema_calval_dev = 'calval_ref_data'
    query_template_calval_dev = """

with 
Hourly_data_alp as
	( select alp_h.time,alp_h.device,alp_h.location,alp_h.tair,alp_h.p,alp_h.swdw,alp_h.tdew,alp_h.swuw,alp_h.lwdw,alp_h.lwuw,alp_h.lfw,
	 		alp_h.tbelow,alp_h.pardw,alp_h.paruw,alp_h.lat,alp_h.long,alp_h.rh,
		alp_h.precip from device_data.hourly_alp alp_h				
		where time between '{start}' and '{end}'
			--alp_h.device=case when pdevice is null then alp_h.device else pdevice end 
	),
calvaldata as
(select crd.ref_hourlydate,cdr.mark2 device,avg(crd.ref_tair) ref_tair,avg(crd.ref_p) ref_p,
					avg(crd.ref_swdw) ref_swdw,avg(crd.ref_tdew) ref_tdew,
					avg(crd.ref_swuw) ref_swuw,avg(crd.ref_lwdw) ref_lwdw,
					avg(crd.ref_lwuw) ref_lwuw,avg(crd.ref_lfw) ref_lfw,
					avg(crd.ref_tbelow) ref_tbelow,avg(crd.ref_pardw) ref_pardw,
					avg(crd.ref_paruw) ref_paruw,avg(crd.ref_lat) ref_lat,avg(crd.ref_long) ref_long,
					avg(crd.ref_rh) ref_rh,
					sum(crd.ref_precip) ref_precip
from device_data.calval_ref_data crd 
join device_data.calvaldevicereference cdr 	 on crd.site_id=cdr.site
where crd.ref_hourlydate between '{start}' and '{end}'
	--	and cdr.mark2=case when null is null then cdr.mark2 else null end
		and crd.ref_precip is not null
	group by crd.ref_hourlydate,cdr.mark2
)	
-- currently only pull tair/ref_tair, you can expand the columns to fit your need	
select cv.ref_hourlydate as time, cv.device,alph.tair,cv.ref_tair
from Hourly_data_alp alph 
join calvaldata cv on alph.time=cv.ref_hourlydate and alph.device=cv.device				
where cv.device = '{device}'
order by cv.device, cv.ref_hourlydate asc 


"""

    sql_query = query_template_calval_dev.format(schema=schema_calval_dev, device=device, start=begin, end=end)

    df = pd.read_sql_query(sql_query, cnx)

    return df

In [76]:
device_list=['C004223', 'C005042'] 
start='2021-05-21'
end='2021-06-17'

In [77]:
series = []
df_final = pd.DataFrame()

for device in device_list:
    print(device, start, end)
#     device='*'
    df_calval_dev = read_calval_dev(pg_conn, device, start, end)
    print(df_calval_dev.shape)
    if not df_calval_dev.empty:
        df_final=pd.concat([df_final, df_calval_dev])


C004223 2021-05-21 2021-06-17
(649, 4)
C005042 2021-05-21 2021-06-17
(649, 4)


In [78]:
#change set time data as index, run 
df_calval_dev=settimeindex(df_final)

In [79]:
df_calval_dev

,index,time,device,tair,ref_tair
time,,,,,
2021-05-21 00:00:00+00:00,0,2021-05-21 00:00:00+00:00,C004223,22.731041,22.444444
2021-05-21 01:00:00+00:00,1,2021-05-21 01:00:00+00:00,C004223,22.362041,22.129629
2021-05-21 02:00:00+00:00,2,2021-05-21 02:00:00+00:00,C004223,22.492142,22.638888
2021-05-21 03:00:00+00:00,3,2021-05-21 03:00:00+00:00,C004223,22.664692,22.787038
2021-05-21 04:00:00+00:00,4,2021-05-21 04:00:00+00:00,C004223,22.091541,22.583333
...,...,...,...,...,...
2021-06-16 20:00:00+00:00,644,2021-06-16 20:00:00+00:00,C005042,25.710726,24.379629
2021-06-16 21:00:00+00:00,645,2021-06-16 21:00:00+00:00,C005042,26.135440,25.106482
2021-06-16 22:00:00+00:00,646,2021-06-16 22:00:00+00:00,C005042,26.936916,25.532408


## Here We learn how to pull CalVal and Device Data, which used to be tedious procedure! 

### Q: can you modify the query to pull a CALVAL Mark Device for a time window of your interest?

## Great Job!

In [ ]:
#